## Counting Words, Part 1: TF/IDF ##

*Based of tutorials by [Matthew Lavin](https://programminghistorian.org/en/lessons/analyzing-documents-with-tfidf) and [Kavita Ganesan](https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XZVlcOdKhSw)*

Topic modelling can be a powerful technique for identifying the themes that recur in a corpus. But in many cases, just counting words can also tell you a lot. 

To begin, we're going to explore a method called Term Frequency - Inverse Document Frequency (tf-idf). Tf-idf comes up a lot in text analysis projects because it’s both a corpus exploration method and a pre-processing step for many other text-mining measures and models.

The procedure was introduced in a 1972 paper by Karen Spärck Jones under the name “term specificity,” and the basic idea is this:

Instead of representing a term in a document by its raw frequency (its number of occurrences) or its relative frequency (the term count divided by the document length), each term is *weighted* by dividing the term frequency by the number of documents in the corpus containing the word. 

The overall effect of this weighting scheme is to avoid a common problem when conducting text analysis: the most frequently used words in a document are often the most frequently used words in all of the documents. We encountered this very problem in our topic model of the CCP Corpus.

By contrast, terms with the highest tf-idf scores are the terms in a document that are distinctively frequent in a document, when that document is compared other documents. When you sort by tf-idf score, these distinctive terms rise to the top. 

### TF-IDF: How to do it ###

To start, create a list in which each doc is a single string. Note that this is nearly the same as our iter_docs function from last class. You'll get very familiar with writing document and text pre-processing code like this by the end of the class.

In [ ]:
import os

base_dir = "./2019-09-ccp-corpus-0.3/ccprecords/"

all_docs = []

docs = os.listdir(base_dir)

for doc in docs:
    if not doc.startswith('.'): # get only the .txt files
        with open(base_dir + doc, "r") as file:
            text = file.read()
            all_docs.append(text)

# just take a look at the first item to be sure
all_docs[0]

You can  calculate tf-idf manually, but we'll use scikit-learn's tf-idf modules because they have built-in tokenization. 

Note also that we're encountering a *third* library that does tokenization for us. The takeaway is that after getting your text into whatever format is required, the next step us (usually) to tokenize it.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#instantiate CountVectorizer()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(all_docs)

# check shape
word_count_vector.shape


In [ ]:
# we can look at the whole vocabulary and counts like this
cv.vocabulary_


In [ ]:
# and we can sort it like this:

sum_words = word_count_vector.sum(axis=0) # sum_words is a vector that contains
                                            # the sum of each word occurrence in all 
                                            # texts in the corpus. In other words, 
                                            # we are adding the elements for each column of
                                            # the word_count_vector matrix

# then sort the list of tuples that contain the word and their occurrence in the corpus.
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

# display the top 10
words_freq[:10]

Raw word frequencies: not that interesting! 

So now let's calculate the IDF values.

In [ ]:
import pandas as pd # this will help us keep track of our data; 
# we'll talk about pandas and dataframes in more detail on Tuedsay

# Call tfidf_transformer.fit on the word count vector we computed earlier.

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

But what are these weights more precisely?

The most direct formula would be **N/df<sub>i</sub>**, where N represents the total number of documents in the corpus, and df is the number of documents in which the term appears. 

However, many implementations of tf-idf, including scikit-learn, normalize the results with additional operations. 

In tf-idf, normalization is generally used in two ways, and for two reasons: first, to prevent bias in term frequency from terms in shorter or longer documents; and second, as above, to calculate each term’s idf value. 


Scikit-learn’s implementation of tf-idf represents N as **N+1**, calculates the natural logarithm of **(N+1)/df<sub>i</sub>**, and then adds **1** to the final result. 

![tf-idf](http://lklein.lmc.gatech.edu/wp-content/uploads/2019/10/Screen-Shot-2019-10-02-at-11.52.31-PM.png)

In any case, one you have the IDF values, you can now compute the tf-idf scores for any document or set of documents. Let’s compute tf-idf scores for the documents in our corpus.

In [ ]:
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(word_count_vector)

Now, let’s print the tf-idf values of the first document to see if it makes sense. What we are doing below is, placing the tf-idf scores from the first document into a pandas data frame and sorting it in descending order of scores.

In [ ]:
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores for the first doc
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

Notice that only certain words have scores. This is because all the words in this document have a tf-idf score and everything else shows up as zeroes.

Also, there are still some very common words that are evidently distinctive, but they're also just not that interesting. 

So now we're going to do it again with scikit-learn's stopword list. And since we're tf-idf pros, we're going to use scikit-learn's all-in-one TF-IDF vectorizer. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(stop_words='english', use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(all_docs)

In [ ]:
# as above, get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

Finally, let's store our TF-IDF vectors to files so that we can use them next class.

In [ ]:
base_dir = "./2019-09-ccp-corpus-0.3/ccprecords/"

# make a directory to store them in
os.mkdir("./tf_idf_output")

docs = os.listdir(base_dir)

csvs = []

for doc in docs:
    if not doc.startswith('.'): # get only the .txt files
        csv = doc.replace(".txt",".csv")
        csvs.append(csv)

# convert sparse matrix to array
tfidf_vectors_as_array = tfidf_vectorizer_vectors.toarray()

# loop each item in tfidf_vectors_as_array, 
for counter, doc in enumerate(tfidf_vectors_as_array): # note enumerate. useful! 
    # construct a dataframe
    tf_idf_tuples = list(zip(tfidf_vectorizer.get_feature_names(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)

    # output to a csv using the enumerated value for the filename
    one_doc_as_df.to_csv(csvs[counter])


